LIBRARIES

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, metrics
import pandas as pd
from sklearn.ensemble import RandomForestRegressor




import matplotlib.dates as mdates
import seaborn as sns
from sklearn import preprocessing

In [2]:
df=pd.read_csv('../Data/clean_FB.csv')
print(df.head())

                         Time  Protocol  Length        DT  source port  \
0  2021-12-12 19:24:54.113588        10      66  0.147364        443.0   
1  2021-12-12 19:24:54.215187        12     281  0.101599        443.0   
2  2021-12-12 19:24:57.467081        10      66  0.151991        443.0   
3  2021-12-12 19:24:57.629709        12     283  0.162628        443.0   
4  2021-12-12 19:24:58.846692        10      66  0.011171        443.0   

     Absolute Time  Cumulative Bytes  
0  00:24:54.113588               894  
1  00:24:54.215187              1175  
2  00:24:57.467081              1727  
3  00:24:57.629709              2010  
4  00:24:58.846692              2336  


ALGORITHM

In [3]:
x=df[['Length','DT','Cumulative Bytes']].values
y=df['source port'].values

In [4]:
# create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
  
# fit the regressor with x and y data
regressor.fit(x, y) 

RandomForestRegressor(random_state=0)

CAMPARING

In [ ]:
label_encoder = preprocessing.LabelEncoder()

df5=pd.read_csv('../Data/Youtube.csv')
df5.drop('No.',axis=1,inplace=True)




df5=df5[~df5['source port'].isnull()].copy()
df5['Protocol']=label_encoder.fit_transform(df5['Protocol'])
#onehot-encoding
df5.isnull().sum()
#df.drop('Frequency',axis=1,inplace=True)
#df.drop('TX Delay',axis=1,inplace=True)

print(df5.head())